In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as UtiData
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import cv2
import numpy as np

import os, csv, argparse
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from fcn32s import VGG16_FCN32s
from utils import *
from mean_iou_evaluate import *
from dataset import *
from criterion import *

In [ ]:
VGG16_net = torchvision.models.vgg16(pretrained=True).cuda()

In [ ]:
n_class = 7
batch_size = 6
MAX_EPOCHS = 20
cuda_device = 0

net = VGG16_FCN32s(n_class).cuda()
net.copy_vgg16(VGG16_net)
del VGG16_net
#net = torch.nn.DataParallel(net).cuda()

filepath = 'dataset/train'
train_loader = get_data_loader(filepath, batch_size)

criterion = CrossEntropyLoss2d().cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)

for ep in range(0, MAX_EPOCHS):
    epoch_loss = 0
    for it, (images, labels) in enumerate(train_loader):
        assert images.size(0) == labels.size(0)
        images = Variable(images.cuda()).cuda()
        labels = Variable(labels.cuda()).cuda()
        output = net(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        #print('epoch: {}, it: {}, loss: {}'.format(ep, it, loss.data[0]))
        epoch_loss += loss.data[0]
    print(epoch_loss/it)

/home/pixiym/DLCV/hw3/criterion.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.nll_loss(F.log_softmax(inputs), targets)


1.6182069259804566
1.4610129593254684
1.128974515896339
1.0895059977258954
1.118976782978355
1.1571483493625343
1.2911657885297552
1.344711133953813
1.4271697895867483
